<a href="https://colab.research.google.com/github/WHU-Peter/COMP6248-Deep-Learning/blob/master/4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 5.4MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-0.0068,  0.3949, -0.2448,  ...,  0.4216, -0.3206, -3.4627],
        [ 0.2027,  2.2052,  0.5469,  ...,  1.5932,  0.2149, -0.1240],
        [ 0.1061, -1.5596,  0.0578,  ..., -0.1514, -0.1357, -0.5132],
        ...,
        [ 0.0685,  3.3276,  0.0321,  ..., -0.0697,  1.2549,  0.8131],
        [ 0.0040, -1.1185,  2.3739,  ...,  0.2817,  0.4853, -0.7489],
        [-0.2416,  0.5045,  1.5618,  ...,  0.0103, -0.7677, -0.9165]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [3]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 0.0457, -1.1144,  0.0719,  ...,  0.0453,  0.2829, -2.6503],
        [-0.2066, -0.1251, -0.2353,  ..., -0.6188, -0.0091,  1.1953],
        [ 0.0503, -0.1861,  0.5681,  ..., -0.0177,  0.4772,  0.0369],
        ...,
        [ 0.0052,  0.3737,  0.0328,  ...,  0.0043,  1.0702, -0.1867],
        [ 0.0412,  0.6291,  2.5113,  ..., -0.3417, -0.8863,  0.4178],
        [ 1.5622,  0.2624,  0.1066,  ..., -0.4508,  0.1885,  0.8579]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 2.6423e-01,  1.4758e+00,  1.4869e+00,  ...,  9.3092e-01,
         -1.1443e-01,  7.9350e-01],
        [-2.1567e-03,  1.1868e+00,  2.7799e+00,  ...,  8.0838e-01,
         -1.5047e+00,  2.5765e-02],
        [ 1.2789e+00, -5.3422e-01, -3.4984e-02,  ...,  5.8232e-01,
         -2.6975e-01,  1.7367e+00],
        ...,
        [-3.0704e-01, -1.8118e+00, -4.4199e-02,  ...,  6.1634e-01,
          2.0787e+00, -5.1776e-01],
        [-6.6133e-01,  1.7138e+00,  4.2517e-01,  ..., -9.4308e-01,
          1.0251e+00, -1.6494e-01],
        [-6.6503e-02,  2.6814e-02, -5.9387e-02,  ...,  1.5811e-01,
          3.9467e-01, -3.1370e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

GPU核比较多，天生支持计算密集型程序

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

矩阵大小 （不太确定）